<h2>Lab 4</h2>
<br>
In this lab you are going to develop a kernel in the FPGA to invert the colors of an image.

Load image.

In [2]:
from PIL import Image as PIL_Image
import numpy
import time

# load and display the image
imgOrig = PIL_Image.open("image0.jpg")
imgWidth, imgHeight = imgOrig.size
display(imgOrig)
PADSIZE=8
BLK_HEIGHT=128
BLK_WIDTH=128
# convert image to numpy array with dimensions imgHeight x imgWidth x 3
imgArr = numpy.asarray(imgOrig)
imgHeight, imgWidth, num_channels = imgArr.shape

if imgHeight < BLK_HEIGHT or imgWidth < BLK_WIDTH: #pad for smaller images
  pad_height = max(0, BLK_HEIGHT - imgHeight)
  pad_width = max(0, BLK_WIDTH - imgWidth)

  pad_top = pad_height // 2
  pad_bottom = pad_height - pad_top
  pad_left = pad_width // 2
  pad_right = pad_width - pad_left

  padding = ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)) if num_channels > 1 else ((pad_top, pad_bottom), (pad_left, pad_right))
  imgArr = np.pad(imgArr, padding, mode='edge')

paddedimgArr = np.pad(imgArr, ((PADSIZE, PADSIZE), (PADSIZE, PADSIZE), (0, 0)), mode='edge') #pad edges with nearest neighbour

FileNotFoundError: [Errno 2] No such file or directory: 'image0.jpg'

Compute the inverted image in software (Python).

Compute the inverted image using the overlay.

In [ ]:
from pynq import Overlay
from pynq.mmio import MMIO

print('Inverting image using hardware acceleration ...')

# load overlay
overlay = Overlay('/home/xilinx/jupyter_notebooks/lab4/invert.bit')
start_time = time.time()

# address configuration
ADDR_BASE_IN=0x40000000
ADDR_RANGE_IN=0x4000
ADDR_BASE_OUT=0x40004000
ADDR_RANGE_OUT=0x4000

# get MMIO handlers
inMMIO = MMIO(ADDR_BASE_IN, ADDR_RANGE_IN)
outMMIO = MMIO(ADDR_BASE_OUT, ADDR_RANGE_OUT)

# view linear memory areas as 128x128 matrices
# inMat and outMat are numpy.ndarray elements with dimensions 64x64
BLK_UHEIGHT=BLK_HEIGHT-2*PADSIZE
BLK_UWIDTH=BLK_WIDTH-2*PADSIZE

inMat = numpy.reshape(inMMIO.array,(BLK_HEIGHT,BLK_WIDTH))
outMat = numpy.reshape(outMMIO.array,(BLK_UHEIGHT,BLK_UWIDTH))
imgArrAccel=numpy.full([imgHeight, imgWidth, 3], dtype=numpy.uint8, fill_value=0x00)

# run BLK_HEIGHT x BLK_WIDTH slices of the image through the accelerator
# the slices only shift BLK_UHEIGHT x BLK_UWIDTH
for offsetRow in range(0,imgHeight,BLK_UHEIGHT):
    if offsetRow + BLK_HEIGHT > imgHeight:
        offsetRow = imgHeight - BLK_HEIGHT
    for offsetCol in range(0,imgWidth,BLK_UWIDTH):
        if offsetCol + BLK_WIDTH > imgWidth:
            offsetCol = imgWidth - BLK_WIDTH
        # write the BLK_HEIGHT x BLK_WIDTH slice to inMat
        for row in range(0,BLK_HEIGHT,1):
            for col in range(0,BLK_WIDTH,1):
                i = offsetRow + row
                j = offsetCol + col
                inMat[row,col] = (imgArr[i,j,0] << 16) | (imgArr[i,j,1] << 8) | imgArr[i,j,2]
        # start the hardware kernel and wait for it to complete
        overlay.invert_0.register_map.CTRL.AP_START=1
        while overlay.invert_0.register_map.CTRL.AP_DONE == 0:
            pass
        # read the BLK_UHEIGHT x BLK_UWIDTH slice from outMat
        for row in range(0,BLK_UHEIGHT,1):
            for col in range(0,BLK_UWIDTH,1):
                i = offsetRow + row
                j = offsetCol + col
                imgArrAccel[i,j] = [ 0xff & (outMat[row,col] >> 16), 0xff & (outMat[row,col] >> 8), 0xff & outMat[row,col] ]

imgAccel = PIL_Image.fromarray(imgArrAccel)
end_time = time.time()
execution_time = end_time - start_time
print(f'Done. Run time: {execution_time}s.')
display(imgAccel)